# Run behavior cloning

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import time
import torch
import numpy as np

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxend module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = False
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController
from sensorprocessing.sp_helper import get_transform_to_sp
from sensorprocessing.sp_factory import create_sp
from bc_factory import create_bc_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

Approxend module not found, cannot use gamepad
***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
Using device: cuda


In [2]:
experiment = "behavior_cloning"
run = "bc_run_00"

exp = Config().get_experiment(experiment, run)

exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])
exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])



***ExpRun**: Configuration for exp/run: behavior_cloning/bc_run_00 successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/robot_position_controller successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\experiment_configs\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_mlp_00 successfully loaded
***ExpRun**: Configuration for exp/run: sensorprocessing_conv_vae/sp_vae_256 successfully loaded


## Initialize the controllers

In [ ]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path))   

cap0 works


C:\Users\lboloni\AppData\Local\Temp\ipykernel_133288\3986934847.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
last_interval = 0
controller_interval = 0.1

# the backlog of the last sequence length steps
z_back = []

while True:
    start_time = time.time() 

    # Let us assume that the first image here is the one
    image = camera_controller.images[0]
    # FIXME: this is totally won't work like this
    # there should be a notebook with the input processing???
    sensor_readings = image
    z = sp.process(sensor_readings)
    z_back.append(z)
    if exp_bc["sequence_length"]: # no sequence processsing
        if len(z_back) < exp_bc["sequence_length"]:
            ## we don't have enough in the queue, wait.
            print("Not acting, waiting on the queue")
            pass
        else:
            # FIXME: this should be applied online, not step by step
            input_list = z_back[-exp_bc["sequence_length"]:]
            input_nd = np.array(input_list)
            z_tensor = torch.tensor(input_nd).unsqueeze(0).to(device)
    else:
        z_tensor = torch.tensor(input_nd).unsqueeze(0).to(device)
    if not model.stochastic:                
        a_pred = model(z_tensor)
    else:
        a_pred = model.forward_and_sample(z_tensor)

    # move the robot
    pos_target = PositionController.from_normalized_vector(a_pred, exp_robot_controller)
    robot_controller.move(pos_target)

    # ensuring that the execution happens 
    end_time = time.time() 
    execution_time = end_time - start_time 
    last_interval = execution_time
    time_to_sleep = max(0.0, controller_interval - execution_time) 
    time.sleep(time_to_sleep) 
